In [ ]:
import psycopg2
import sys
import re
 
def main():
    #Define our connection string
    conn_string = "host='52.43.75.12' port='5432' dbname='ndexstats' user='ndexstats' password='dashboard101'"
 
    # print the connection string we will use to connect
    print "Connecting to database\n\t->%s" % (conn_string)
 
    # get a connection, if a connect cannot be made an exception will be raised here
    try:
        conn = psycopg2.connect(conn_string)
    except:
        e = sys.exc_info()[0]
        print "Error: %s" % e 
        sys.exit()

        
    # conn.cursor will return a cursor object, you can use this cursor to perform queries
    cursor = conn.cursor()
    print "Connected!\n"
    
    # input log file to be parsed and inserted into ndexstats table -- hardcoded for now
    # inputfile = "ndex-2016-07-20.0.log"
    # inputfile = "test1.log"
    inputfile = "ndex-2016-07-20.0.log"
    
    number_of_lines_processed = 0
    number_of_lines_total = 0
    
    beginning_of_log_regex = re.compile(r"^\[\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}\]")
    
    with open(inputfile, "r") as f:
        for line in f:
            
            number_of_lines_total += 1
            
            if (not beginning_of_log_regex.search(line)) :
                continue
            
            split_line = re.split(r'\t+', line)           
            
            log_entry_split = [token.replace('[','').replace(']','').replace('\n','') for token in split_line]  
            #print log_entry_split, 'length=', len(log_entry_split)
            timestamp, type, ip, id, account, log, api = log_entry_split
            
            # skip logs without Transaction IDs and DEBUG log entries 
            if (not id) or (type == 'DEBUG') :
                continue
                   
            start = log.startswith('start:')
            end = log.startswith('end:')
            
            insertOrUpdateRecordInTable(cursor, timestamp, type, ip, id, account, log, api, start, end)
            
            number_of_lines_processed += 1
          
    print "\n finished inserting,  now calculate ..."

    # use transaction_start and transaction_end to calculate and fill transacation_duration_in_ms column
    conn.commit()
    calculateTransactionsDurations(cursor)
    
    print "\nnumber of lines total =", number_of_lines_total, ", processed =", number_of_lines_processed
    
    conn.commit()
    countAll(cursor)
        
    cursor.close()
    conn.close()
    
    
def countAll(cursor):
    cursor.execute("select count(*) from ndexstats ")
    number_of_records = cursor.fetchone()[0]
    
    print "number_of_records = ", number_of_records
 
 
         
def insertOrUpdateRecordInTable(cursor, timestamp, type, ip, id, account, log_entry, api, start, end):
    year, month, day, hour, min, sec, id1, id2 = re.split('-', id) 
    
    if (start):
        query = "insert into ndexstats (transaction_id, transaction_start, transaction_type, client_ip, account, " + \
            "api, log_entry_start, transaction_year, transaction_month, transaction_day, " + \
            "transaction_hour, transaction_min, transaction_sec) "
        
        query += " values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) on conflict(transaction_id) do nothing"
        data = (id, timestamp, type, ip, account, api, log_entry, year, month, day, hour, min, sec)
    
    elif (end or type=='ERROR'):
        query = "update ndexstats set (transaction_end, log_entry_end, transaction_type) = "
        query += "(%s, %s, %s) where transaction_id = %s"
        data  = (timestamp, log_entry, type, id)
        
    else:
        return
    
    try:
        cursor.execute(query, data)
    except:
        print "error; id=", id
        #e = sys.exc_info()[0]
        #print "Error: %s" % e 
        #print data
    
    
def calculateTransactionsDurations(cursor):
    query = "select transaction_id, \
               extract(epoch from to_timestamp(transaction_start,'YYYY-MM-DD HH24:MI:SS,MS'))*1000 as transaction_start, \
               extract(epoch from to_timestamp(transaction_end,'YYYY-MM-DD HH24:MI:SS,MS'))*1000 as transaction_end, \
               transaction_duration_in_ms \
            from ndexstats where transaction_duration_in_ms is null"
    
    cursor.execute(query)
    

    count = 0
    query = "update ndexstats set transaction_duration_in_ms = %s where transaction_id = %s"
    
    rows = cursor.fetchall()

    for row in rows:
      
        transaction_start = row[1];
        transaction_end = row[2];
        
        if (transaction_start is None or transaction_end is None):
            transaction_duration_in_ms = -1
        else:
            transaction_duration_in_ms = long(row[2]) - long(row[1])
        
        data  = (transaction_duration_in_ms, row[0])

        try:
            cursor.execute(query, data)
            
        except:
            e = sys.exc_info()[0]
            print "Error: %s" % e 
        
        count += 1

    print "updated ", count, " records"  


if __name__ == "__main__":
    main()